# CNN Model Training for Climbing Grade Prediction
This notebook trains a Convolutional Neural Network to predict climbing grades from hold placements and route features.

In [ ]:
# Cell 1: Setup and Imports
import sys
import os
sys.path.append('.')  # Add current directory to Python path

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

# Simple matplotlib settings
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
%matplotlib inline

print("✓ Imports successful!")
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

In [ ]:
# Cell 2: Load Custom Modules
try:
    from src.data.preprocessing import load_data, create_boulder_angle_dataframe
    from src.data.analysis import analyze_and_clean_data
    from src.models.preprocessing import create_train_test_split
    from src.models.cnn_model import create_cnn_model
    from src.visualization.model_plots import plot_training_history
    from src.features.grade_conversion import difficulty_to_vgrade
    print("✓ Custom modules loaded successfully!")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Make sure you're running this notebook from the project root directory")

In [ ]:
# Cell 3: Load and Prepare Data
print("Loading climbing data...")
data_path = "data/raw/climbs.csv"

if os.path.exists(data_path):
    # Load raw data
    df = load_data(data_path)
    print(f"✓ Loaded {len(df)} climbing routes")
    
    # Clean the data
    print("\nCleaning data...")
    cleaned_df, profile, stats = analyze_and_clean_data(df, 
                                                       upper_percentile=95, 
                                                       save_cleaned=False)
    
    if cleaned_df is not None:
        print(f"✓ Data cleaned: {len(cleaned_df)} routes remain")
        
        # Create boulder angle dataframe
        print("\nCreating angle-specific dataset...")
        boulder_angles_df = create_boulder_angle_dataframe(cleaned_df, min_ascents=2)
        print(f"✓ Created {len(boulder_angles_df)} angle-specific entries")
        
        # Show sample of the data
        print("\nSample of processed data:")
        display(boulder_angles_df[['name', 'angle', 'grade', 'ascents', 'hold_count']].head(10))
        
    else:
        print("❌ Failed to clean data")
else:
    print(f"❌ Data file not found at {data_path}")
    print("Please make sure your climbs.csv file is in the data/raw/ directory")

In [ ]:
# Cell 4: Data Statistics and Visualization
if 'boulder_angles_df' in locals():
    print("Dataset Statistics:")
    print(f"Total entries: {len(boulder_angles_df)}")
    print(f"Unique problems: {boulder_angles_df['name'].nunique()}")
    print(f"Grade range: {boulder_angles_df['grade'].min():.1f} - {boulder_angles_df['grade'].max():.1f}")
    print(f"Angle range: {boulder_angles_df['angle'].min()}° - {boulder_angles_df['angle'].max()}°")
    print(f"Hold count range: {boulder_angles_df['hold_count'].min()} - {boulder_angles_df['hold_count'].max()}")
    
    # Visualize grade distribution
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Grade distribution
    axes[0,0].hist(boulder_angles_df['grade'], bins=30, alpha=0.7, edgecolor='black')
    axes[0,0].set_title('Grade Distribution (Difficulty Scale)')
    axes[0,0].set_xlabel('Difficulty Score')
    axes[0,0].set_ylabel('Frequency')
    
    # Angle distribution
    axes[0,1].hist(boulder_angles_df['angle'], bins=20, alpha=0.7, edgecolor='black', color='orange')
    axes[0,1].set_title('Angle Distribution')
    axes[0,1].set_xlabel('Board Angle (degrees)')
    axes[0,1].set_ylabel('Frequency')
    
    # Hold count distribution
    axes[1,0].hist(boulder_angles_df['hold_count'], bins=25, alpha=0.7, edgecolor='black', color='green')
    axes[1,0].set_title('Hold Count Distribution')
    axes[1,0].set_xlabel('Number of Holds')
    axes[1,0].set_ylabel('Frequency')
    
    # Grade vs Hold Count scatter
    axes[1,1].scatter(boulder_angles_df['hold_count'], boulder_angles_df['grade'], alpha=0.5)
    axes[1,1].set_title('Grade vs Hold Count')
    axes[1,1].set_xlabel('Number of Holds')
    axes[1,1].set_ylabel('Difficulty Score')
    
    plt.tight_layout()
    plt.show()
    
    # Show V-grade distribution
    v_grades = [difficulty_to_vgrade(grade) for grade in boulder_angles_df['grade']]
    v_grade_counts = pd.Series(v_grades).value_counts().sort_index()
    
    plt.figure(figsize=(12, 6))
    v_grade_counts.plot(kind='bar', color='skyblue', edgecolor='black')
    plt.title('Distribution by V-Grade')
    plt.xlabel('V-Grade')
    plt.ylabel('Number of Problems')
    plt.xticks(rotation=45)
    plt.grid(axis='y', alpha=0.3)
    plt.show()

In [ ]:
# Cell 5: Create Train-Test Split
if 'boulder_angles_df' in locals():
    print("Creating train-test split...")
    
    # Create the split (ensures same boulder problems don't appear in both sets)
    X_train, X_test, y_train, y_test = create_train_test_split(boulder_angles_df, 
                                                              test_size=0.2, 
                                                              random_state=42)
    
    print(f"\n✓ Data split complete:")
    print(f"Training set: {len(X_train)} samples")
    print(f"Test set: {len(X_test)} samples")
    print(f"Training problems: {X_train['name'].nunique()}")
    print(f"Test problems: {X_test['name'].nunique()}")
    
    # Check grade distribution in splits
    train_v_grades = [difficulty_to_vgrade(grade) for grade in y_train]
    test_v_grades = [difficulty_to_vgrade(grade) for grade in y_test]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    pd.Series(train_v_grades).value_counts().sort_index().plot(kind='bar', ax=ax1, title='Training Set V-Grades')
    pd.Series(test_v_grades).value_counts().sort_index().plot(kind='bar', ax=ax2, title='Test Set V-Grades')
    
    ax1.set_ylabel('Count')
    ax2.set_ylabel('Count')
    ax1.tick_params(axis='x', rotation=45)
    ax2.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ No data available for splitting")

In [ ]:
# Cell 6: Train the CNN Model
if all(var in locals() for var in ['X_train', 'X_test', 'y_train', 'y_test']):
    print("Starting CNN model training...")
    print("This may take several minutes depending on your hardware.\n")
    
    try:
        # Train the model
        model, history, metrics = create_cnn_model(
            boulder_angles_df, 
            X_train, X_test, 
            y_train, y_test
        )
        
        print("\n" + "="*50)
        print("🎉 MODEL TRAINING COMPLETE!")
        print("="*50)
        
        # Display metrics
        print("\nFinal Model Performance:")
        for metric_name, value in metrics.items():
            print(f"  {metric_name.replace('_', ' ').title()}: {value:.4f}")
            
        training_complete = True
        
    except Exception as e:
        print(f"❌ Error during model training: {str(e)}")
        print("\nTroubleshooting tips:")
        print("- Make sure TensorFlow is properly installed")
        print("- Check if you have enough memory (reduce batch size if needed)")
        print("- Verify your data has the required columns")
        training_complete = False
        
else:
    print("❌ Training data not available. Please run the previous cells first.")
    training_complete = False

In [ ]:
# Cell 7: Visualize Training Results
if 'training_complete' in locals() and training_complete and 'history' in locals():
    print("Plotting training history...")
    
    # Plot training history
    try:
        fig = plot_training_history(history)
        plt.show()
        
        # Additional detailed plots
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Loss over time
        epochs = range(1, len(history.history['loss']) + 1)
        axes[0,0].plot(epochs, history.history['loss'], 'b-', label='Training Loss')
        axes[0,0].plot(epochs, history.history['val_loss'], 'r-', label='Validation Loss')
        axes[0,0].set_title('Training and Validation Loss')
        axes[0,0].set_xlabel('Epochs')
        axes[0,0].set_ylabel('Loss')
        axes[0,0].legend()
        axes[0,0].grid(True, alpha=0.3)
        
        # MAE over time
        axes[0,1].plot(epochs, history.history['mean_absolute_error'], 'b-', label='Training MAE')
        axes[0,1].plot(epochs, history.history['val_mean_absolute_error'], 'r-', label='Validation MAE')
        axes[0,1].set_title('Training and Validation MAE')
        axes[0,1].set_xlabel('Epochs')
        axes[0,1].set_ylabel('Mean Absolute Error')
        axes[0,1].legend()
        axes[0,1].grid(True, alpha=0.3)
        
        # Learning rate (if available)
        if 'lr' in history.history:
            axes[1,0].plot(epochs, history.history['lr'], 'g-')
            axes[1,0].set_title('Learning Rate Schedule')
            axes[1,0].set_xlabel('Epochs')
            axes[1,0].set_ylabel('Learning Rate')
            axes[1,0].set_yscale('log')
            axes[1,0].grid(True, alpha=0.3)
        else:
            axes[1,0].text(0.5, 0.5, 'Learning Rate\nNot Recorded', 
                          ha='center', va='center', transform=axes[1,0].transAxes)
            axes[1,0].set_title('Learning Rate Schedule')
        
        # Model performance summary
        axes[1,1].axis('off')
        summary_text = "Model Performance Summary\n\n"
        for metric_name, value in metrics.items():
            summary_text += f"{metric_name.replace('_', ' ').title()}: {value:.4f}\n"
        
        axes[1,1].text(0.1, 0.9, summary_text, transform=axes[1,1].transAxes, 
                      fontsize=12, verticalalignment='top', 
                      bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Error plotting results: {e}")
        
else:
    print("No training results to display. Please run the training cell first.")

In [ ]:
# Cell 8: Make Predictions and Analyze Results
if 'training_complete' in locals() and training_complete and 'model' in locals():
    print("Making predictions on test set...")
    
    try:
        # Get model predictions
        from src.models.cnn_model import create_multichannel_grid, encode_hold_types
        from sklearn.preprocessing import StandardScaler
        
        # Prepare test data
        feature_cols = ['angle', 'hold_count', 'ascents']
        scaler = StandardScaler()
        
        # Scale features (fit on training, transform test)
        X_train_scaled = scaler.fit_transform(X_train[feature_cols])
        X_test_scaled = scaler.transform(X_test[feature_cols])
        
        # Create grids and hold type encodings
        test_grids = np.array([create_multichannel_grid(p) for p in X_test['placements']])
        test_hold_types = np.array([encode_hold_types(p) for p in X_test['placements']])
        test_features = np.hstack((X_test_scaled, test_hold_types))
        
        # Make predictions
        predictions = model.predict([test_grids, test_features])
        
        # Convert to V-grades for interpretation
        actual_v_grades = [difficulty_to_vgrade(grade) for grade in y_test]
        predicted_v_grades = [difficulty_to_vgrade(pred[0]) for pred in predictions]
        
        # Create results DataFrame
        results_df = pd.DataFrame({
            'Actual_Grade': y_test.values,
            'Predicted_Grade': predictions.flatten(),
            'Actual_V_Grade': actual_v_grades,
            'Predicted_V_Grade': predicted_v_grades,
            'Error': y_test.values - predictions.flatten(),
            'Abs_Error': np.abs(y_test.values - predictions.flatten())
        })
        
        print(f"\n✓ Predictions completed for {len(results_df)} test samples")
        
        # Show sample predictions
        print("\nSample Predictions:")
        display(results_df[['Actual_V_Grade', 'Predicted_V_Grade', 'Abs_Error']].head(10))
        
        # Plot predictions vs actual
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        
        # Scatter plot of predictions vs actual
        axes[0].scatter(y_test, predictions, alpha=0.6)
        axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
        axes[0].set_xlabel('Actual Grade')
        axes[0].set_ylabel('Predicted Grade')
        axes[0].set_title('Predictions vs Actual Grades')
        axes[0].grid(True, alpha=0.3)
        
        # Error distribution
        axes[1].hist(results_df['Error'], bins=30, alpha=0.7, edgecolor='black')
        axes[1].axvline(0, color='red', linestyle='--', linewidth=2)
        axes[1].set_xlabel('Prediction Error')
        axes[1].set_ylabel('Frequency')
        axes[1].set_title('Distribution of Prediction Errors')
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # V-grade accuracy analysis
        exact_matches = sum(a == p for a, p in zip(actual_v_grades, predicted_v_grades))
        within_one = sum(abs(actual_v_grades.index(a) - predicted_v_grades.index(p)) <= 1 
                        for a, p in zip(actual_v_grades, predicted_v_grades) 
                        if a in predicted_v_grades and p in actual_v_grades)
        
        print(f"\nV-Grade Prediction Accuracy:")
        print(f"Exact matches: {exact_matches}/{len(actual_v_grades)} ({exact_matches/len(actual_v_grades)*100:.1f}%)")
        print(f"Within ±1 grade: {within_one}/{len(actual_v_grades)} ({within_one/len(actual_v_grades)*100:.1f}%)")
        
    except Exception as e:
        print(f"Error making predictions: {e}")
        import traceback
        traceback.print_exc()
        
else:
    print("No trained model available for predictions. Please run the training cell first.")

In [ ]:
# Cell 9: Save Model and Results
if 'training_complete' in locals() and training_complete and 'model' in locals():
    print("Saving model and results...")
    
    # Create directories
    os.makedirs('models', exist_ok=True)
    os.makedirs('reports', exist_ok=True)
    
    try:
        # Save the model
        model_path = 'models/climbing_cnn_model.h5'
        model.save(model_path)
        print(f"✓ Model saved to {model_path}")
        
        # Save metrics
        metrics_path = 'reports/model_metrics.txt'
        with open(metrics_path, 'w') as f:
            f.write("CNN Model Evaluation Metrics\n")
            f.write("=" * 30 + "\n")
            for key, value in metrics.items():
                f.write(f"{key}: {value:.4f}\n")
            
            # Add training info
            f.write(f"\nTraining Information:\n")
            f.write(f"Training samples: {len(X_train)}\n")
            f.write(f"Test samples: {len(X_test)}\n")
            f.write(f"Total epochs: {len(history.history['loss'])}\n")
            
        print(f"✓ Metrics saved to {metrics_path}")
        
        # Save predictions if available
        if 'results_df' in locals():
            predictions_path = 'reports/test_predictions.csv'
            results_df.to_csv(predictions_path, index=False)
            print(f"✓ Test predictions saved to {predictions_path}")
        
        print("\n🎉 All results saved successfully!")
        
    except Exception as e:
        print(f"Error saving results: {e}")
        
else:
    print("No trained model to save. Please run the training cell first.")

## Summary

This notebook:
1. ✅ Loads and preprocesses climbing route data
2. ✅ Creates train/test splits ensuring no data leakage
3. ✅ Trains a CNN model to predict climbing grades
4. ✅ Evaluates model performance with multiple metrics
5. ✅ Visualizes training progress and results
6. ✅ Saves the trained model and predictions

### Next Steps:
- Try different model architectures
- Experiment with hyperparameter tuning
- Add more features (route setter, popularity, etc.)
- Compare with simpler baseline models
